# Módulo 5: Limpeza e preparação de dados 
## Exercício 1:  Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [179]:
import pandas as pd
import numpy as np
import requests

# 1) carregar base e verificar linhas duplicadas
sinasc = pd.read_csv('SINASC_RO_2019.csv')

antes_drop = sinasc.shape
sinasc.drop_duplicates().shape
depois_drop = sinasc.shape

if antes_drop == depois_drop:
    print(f'Não há linhas duplicadas {antes_drop}')
else:
    print(f'Há linhas duplicadas ({antes_drop}-{depois_drop})')

Não há linhas duplicadas (27028, 69)


In [180]:
df_sinasc = pd.DataFrame(sinasc)
df_sinasc.head(5)

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025


In [181]:
# 2) missing por variável
df_sinasc.isna().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [182]:
# 3) filtrar campos
campos = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
                     'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
df_selecao = df_sinasc[campos]
df_selecao.head(5)

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0


In [183]:
df_selecao.shape

(27028, 9)

In [184]:
df_selecao.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [185]:
# 4) apagar "apgar5" não preenchido

df_selecao = df_selecao.dropna(subset=['APGAR5'])

df_selecao.shape



(26925, 9)

In [186]:
df_selecao.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [187]:
# 5) Preencher com '9' campos não preenchidos 'ESTCIVMAE' e 'ESCMAE'

df_selecao['ESTCIVMAE'].replace('9', np.nan)
df_selecao = df_selecao.dropna(subset=['ESTCIVMAE'])



In [188]:
df_selecao.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         264
QTDFILVIVO    1537
GESTACAO      1193
GRAVIDEZ        74
CONSULTAS        0
APGAR5           0
dtype: int64

In [189]:
# 6) Preencher com 0 QTDFILVIVO não prenchidos

df_selecao['QTDFILVIVO'].replace('0', np.nan)
df_selecao = df_selecao.dropna(subset=['QTDFILVIVO'])


In [190]:
df_selecao.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         243
QTDFILVIVO       0
GESTACAO      1083
GRAVIDEZ        64
CONSULTAS        0
APGAR5           0
dtype: int64

In [191]:
# 7) Prencher os dados das colunas faltantes

# ESCMAE = escolaridade da mãe preencheremos com '1 a 3 anos'. 
# Se a informação é ignorada por quem informa, ela não é importante para essa pessoa;
# Se ela não é importante, é porque ela não tem. Por isso a escolaridade mínima.

df_selecao['ESCMAE'].replace('1 a 3 anos', np.nan)
df_selecao = df_selecao.dropna(subset=['ESCMAE'])


In [192]:
df_selecao.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE           0
QTDFILVIVO       0
GESTACAO      1060
GRAVIDEZ        64
CONSULTAS        0
APGAR5           0
dtype: int64

In [193]:
# GESTACAO = tempo de gestação de '37 a 41 semanas'. 
# No Sistema SUS evitasse ao máximo o método de cesárea para o nascimento do bebê;
# O corriqueiro é ter parto normal, principalmente em lugares sem infraestrutura como Rondônia;
# Por isso as mulheres tendem a ter o período máximo de gestação.

df_selecao['GESTACAO'].replace('37 a 41 semanas', np.nan)
df_selecao = df_selecao.dropna(subset=['GESTACAO'])

In [194]:
df_selecao.isna().sum()

LOCNASC        0
IDADEMAE       0
ESTCIVMAE      0
ESCMAE         0
QTDFILVIVO     0
GESTACAO       0
GRAVIDEZ      49
CONSULTAS      0
APGAR5         0
dtype: int64

In [195]:
df_selecao['GRAVIDEZ'].value_counts().sort_index()

GRAVIDEZ
Dupla                429
Tríplice e mais        5
Única              23287
Name: count, dtype: int64

In [196]:
# GRAVIDEZ = vamos prencher por idade  = 'Única'; 
# a grande maioria nos dados é Única;

df_selecao['GRAVIDEZ'].replace('Única', np.nan)
df_selecao = df_selecao.dropna(subset=['GRAVIDEZ'])

In [197]:
df_selecao.isna().sum()

LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64

In [200]:
# 8) categorização 'APGAR5'

df_selecao.loc[df_selecao['APGAR5'] <= 3, 'apgar5_cat'] = 'asfixia severa'
df_selecao.loc[(df_selecao['APGAR5'] == 4) | (df_selecao['APGAR5'] == 5), 'apgar5_cat'] = 'asfixia moderada'
df_selecao.loc[(df_selecao['APGAR5'] == 6) | (df_selecao['APGAR5'] == 7), 'apgar5_cat'] = 'asfixia leve'
df_selecao.loc[(df_selecao['APGAR5'] >= 8), 'apgar5_cat'] = 'normal'

In [202]:
df_selecao.head(5)

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,apgar5_cat
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal


In [221]:
frequencia_apgar5 = df_selecao['apgar5_cat'].value_counts()
porcentagem_apgar5 = df_selecao['apgar5_cat'].value_counts()/ len(df_selecao) * 100

df_resultado = pd.concat([frequencia_apgar5, porcentagem_apgar5], axis=1)
print (df_resultado)

                  count      count
apgar5_cat                        
normal            23316  98.292652
asfixia leve        287   1.209898
asfixia severa       64   0.269803
asfixia moderada     54   0.227646


In [223]:
# 9) renomear variáveis

df_resultado.rename(columns={'Frequência': 'frequencia_quantidade', 'Porcentagem': 'porcentagem_porcentagem'}, inplace=True)
df_resultado

,count,count
apgar5_cat,,
normal,23316,98.292652
asfixia leve,287,1.209898
asfixia severa,64,0.269803
asfixia moderada,54,0.227646


In [226]:
frequencia_apgar5 = frequencia_apgar5.rename('Quantidade')
porcentagem_apgar5 = porcentagem_apgar5.rename('Porcentagem')

df_resultado = pd.concat([frequencia_apgar5, porcentagem_apgar5], axis=1)
df_resultado

,Quantidade,Porcentagem
apgar5_cat,,
normal,23316,98.292652
asfixia leve,287,1.209898
asfixia severa,64,0.269803
asfixia moderada,54,0.227646
